# seq2seq: Data Preparation

Abdulhakim Alnuqaydan, Ali Kadhim, Sergei Gleyzer, Harrison Prosper

July 2021

This notebook performs the following tasks:
  1. Read the sequence pairs from __data/seq2seq_data.txt__.
  1. Exclude sequences with complex numbers and with Taylor series expansions longer than 1000 characters.
  1. Write the filtered sequences to __data/seq2seq_data_count.txt__, where count is either 10,000 or 60,000 sequences.
  1. Read filtered data and delimit source (i.e, input) and target (i.e., output) sequences with a tab and newline at the start and end of each sequence, respectively.

In [1]:
import re
import sympy as sp
import numpy as np
import torch

# symbolic symbols
from sympy import exp, \
    cos, sin, tan, \
    cosh, sinh, tanh, ln
x = sp.Symbol('x')

from IPython.display import display
    
# enable pretty printing of equations
sp.init_printing(use_latex='mathjax')

### Filter sequences

In [105]:
of_order    = re.compile(' [+] O[(]x[*][*]5.*[)]')
non_trivial = re.compile(r'(exp|cos|sin|tan|ln|log)[(].*\bx\b')
add_count   = re.compile('_data')
def filterData(inpfile='data/seq2seq_data.txt',
               num_seq=50000,  # number of sequences
               min_len=5,
               max_len=1000):  # maximum length of sequence pairs
    
    data = open(inpfile).readlines()
    
    # eliminate expressions involving complex numbers
    data = filter(lambda d: d.find('I') < 0, data)
    data = list(data)

    # strip away O(...), that is, of order..
    data = [of_order.sub('', d) for d in data]
 
    # split pairs
    data = [ d.split('\t') for d in data ]

    # eliminate expressions that do not involve x, exp, cos etc.
    data = filter(lambda d: 
                  (len(non_trivial.findall(d[0])) > 0) and \
                  (len(non_trivial.findall(d[1])) > 0), data)
    data = list(data)
 
    # keep expressions that are >= min_len characters long
    data = filter(lambda d: 
                  (len(d[0]) >= min_len) and (len(d[1]) >= min_len),
                  data)
    data = list(data)
                  
    # keep expressions that are <= max_len characters long
    data = filter(lambda d: 
                  (len(d[0]) <= max_len) and (len(d[1]) <= max_len), 
                  data)
    data = list(data)
    
    N = min(num_seq, len(data))
    outfile = add_count.sub('_data_%d' % N, inpfile)
    print('output file:', outfile)
    
    data = ['\t'.join(d) for d in data]
    open(outfile, 'w').writelines(data[:N])
    
filterData(num_seq=10000)
filterData(num_seq=42000)

output file: data/seq2seq_data_10000.txt
output file: data/seq2seq_data_42000.txt


### Map sequences to lists of indices

  1. Split data into a train, validation, and test set.
  1. Create a token (i.e., a character) to index map from training data.
  1. Map sequences to arrays of indices.
  1. Implement custom DataLoader.

In [125]:
%%writefile seq2sequtil.py
import numpy as np
import torch
from IPython.display import display

# symbolic symbols
from sympy import Symbol, exp, \
    cos, sin, tan, \
    cosh, sinh, tanh, ln
x = Symbol('x')

class Seq2SeqDataPreparer:
    '''
    This class maps the source (i.e., input) and target (i.e, output) 
    sequences of characters into sequences of indices. The source data 
    are split into x_train, x_valid, and x_test sets and similarly for 
    the target data.
    
    Create a data preparer using
    
    dd = Seq2SeqDataPreparer(X, Y, fractions)
    
    where,

      fractions:    a 2-tuple containing the three-way split of data.
                    e.g.: (40/42, 41/42) means split the data as follows
                    (40000, 1000, 1000)
    '''
    def __init__(self, X, Y,
                 fractions=[40/42, 41/42]): 
        
        self.fractions = fractions
        
        # Get maximum sequence length for input expressions
        self.x_max_seq_len =  max([len(z) for z in X])
        
        # Get maximum sequence length for target expressions
        self.y_max_seq_len =  max([len(z) for z in Y])
        
        # get length of splits into train, valid, test
        N = int(len(X)*fractions[0])
        M = int(len(X)*fractions[1])
        
        # Create token to index map for source sequences
        t = self.token_tofrom_index(X[:N])
        self.x_token2index, self.x_index2token = t
        
        # Create token to index map for target sequences
        t = self.token_tofrom_index(Y[:N])
        self.y_token2index,self.y_index2token = t
        
        # Structure data into a list of blocks, where each block
        # comprises a tuple (x_data, y_data) whose elements have
        #   x_data.shape: (x_seq_len, batch_size)
        #   y_data.shape: (y_seq_len, batch_size)
        #
        # The sequence and batch sizes can vary from block to block.
        
        self.train_data, self.n_train = self.code_data(X[:N], Y[:N])         
        self.valid_data, self.n_valid = self.code_data(X[N:M],Y[N:M])
        self.test_data,  self.n_test  = self.code_data(X[M:], Y[M:])

    def __del__(self):
        pass
    
    def __len__(self):
        n  = 0
        n += self.n_train
        n += self.n_valid
        n += self.n_test
        return n
    
    def __str__(self):
        s  = ''
        s += 'number of seq-pairs (train): %8d\n' % self.n_train
        s += 'number of seq-pairs (valid): %8d\n' % self.n_valid
        s += 'number of seq-pairs (test):  %8d\n' % self.n_test
        s += '\n'
        s += 'number of source tokens:     %8d\n' % \
        len(self.x_token2index)
        s += 'max source sequence length:  %8d\n' % \
        self.x_max_seq_len
        
        try:
            s += '\n'
            s += 'number of target tokens:     %8d\n' % \
            len(self.y_token2index)
            s += 'max target sequence length:  %8d' % \
            self.y_max_seq_len
        except:
            pass

        return s
         
    def num_tokens(self, which='source'):
        if which[0] in ['s', 'i']:
            return len(self.x_token2index)
        else:
            return len(self.y_token2index)
    
    def max_seq_len(self, which='source'):
        if which[0] in ['s', 'i']:
            return self.x_max_seq_len
        else:
            return self.y_max_seq_len
        
    def decode(self, indices):
        # map list of indices to a list of tokens
        return ''.join([self.y_index2token[i] for i in indices])

    def token_tofrom_index(self, expressions):
        chars = set()
        chars.add(' ')  # for padding
        chars.add('?')  # for unknown characters
        for expression in expressions:
            for char in expression:
                chars.add(char)
        chars = sorted(list(chars))
        
        char2index = dict([(char, i) for i, char in enumerate(chars)])
        index2char = dict([(i, char) for i, char in enumerate(chars)])
        return (char2index, index2char)
       
    def get_block_indices(self, X, Y):
        # X, and Y are just arrays of strings.
        #
        # 1. Following Matthew Andrews' suggestion double sort 
        #    expressions, first with targets then sources. But, also
        #    note the ordinal values "i" of the expressions in X, Y.
        sizes = [(len(a), len(b), i) 
                 for i, (a, b) in enumerate(zip(Y, X))]
        sizes.sort()
  
        # 2. Find ordinal values (indices) of all expression pairs 
        #    for which the sources are the same length as are the
        #    targets.
     
        block_indices = []
        n, m, i  = sizes[0] # n, m, i = len(source), len(target), index
        previous = (n, m)
        indices  = [i] # cache index of first expression
        
        for n, m, i in sizes[1:]: # skip first expression
            
            size = (n, m)
            
            if size == previous:
                indices.append(i) # cache index of expression
            else:
                # found a new boundary, so save previous 
                # set of indices...
                block_indices.append(indices)
                
                # ...and start a new list of indices
                indices = [i]

            previous = size
            
        # cache expression indices of last block
        block_indices.append(indices)
        
        return block_indices
    
    def make_block(self, expressions, indices, token2index, unknown):
        
        # batch size of current block
        batch_size = len(indices)
        
        # By construction, all expressions of a block have 
        # the same length
        seq_len = len(expressions[indices[0]])
        
        # Create an empty block of correct shape and size
        data    = np.zeros((seq_len, batch_size), dtype='long')
        #print('seq_len, batch_size: (%d, %d)' % (seq_len, batch_size))
        
        # loop over expressions for current block
        # m: ordinal value of expression in current block
        # k: ordinal value of expression in original list of expressions
        # n: ordinal value of character in a given expression
        
        for m, k in enumerate(indices):
            
            expr = expressions[k]
            
            #print('%5d expr[%d] | %s |' % (m, k, expr[1:-1]))
            
            # copy coded characters to 2D arrays
        
            for n, char in enumerate(expr):
                #print('\t\t(n, m): (%d, %d)' % (n, m))
                try:
                    data[n, m] = token2index[char]
                except:
                    data[n, m] = unknown
                    
        return data
    
    def code_data(self, X, Y):
        # Implement Arvind's idea
        
        # X, Y consist of delimited strings: 
        #   \tab<characters\newline
        
        # loop over sequence pairs and convert them to sequences
        # of integers using the two token2index maps
      
        x_space   = self.x_token2index[' ']
        x_unknown = self.x_token2index['?']
        
        y_space   = self.y_token2index[' ']
        y_unknown = self.y_token2index['?']
 
        # 1. Get boundaries of blocks containing sequences of 
        #    the same length.
        
        block_indices = self.get_block_indices(X, Y)
        
        # 2. Loop over boundaries and create blocks of coded
        #    sequences
        
        blocks = []
        n_data = 0
       
        for indices in block_indices:

            x_data = self.make_block(X, indices, 
                                     self.x_token2index, x_unknown)
 
            y_data = self.make_block(Y, indices, 
                                     self.y_token2index, y_unknown)

            blocks.append((x_data, y_data))
            n = len(indices)
            n_data += n
        
        assert n_data == len(X)
        
        return blocks, n_data
    
class Seq2SeqDataLoader:
    '''
    dataloader = Seq2seqDataLoader(dataset, device, sample=True)    
    '''
    def __init__(self, dataset, device, sample=True):
        self.dataset = dataset
        self.device  = device
        self.sample  = sample  
        self.count   = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        
        # increment iteration counter
        self.count += 1
        
        if self.count <= len(self.dataset):
            
            # 1. randomly pick a block or return blocks in order.
            if self.sample:
                k = np.random.randint(len(self.dataset))
            else:
                k = self.count-1 # must subtract one!
            
            # 2. create tensors directly on the device of interest
            X = torch.tensor(self.dataset[k][0], 
                             device=self.device)
            
            Y = torch.tensor(self.dataset[k][1], 
                             device=self.device)
        
            # shape of X and Y: (seq_len, batch_size)
            return X, Y
        else:
            self.count = 0
            raise StopIteration
        
# Delimit each sequence in filtered sequences
# The start of sequence (SOS) and end of sequence (EOS) 
# tokens are "\t" and "\n", respectively.

def loadData(inpfile):
    # format of data:
    # input expression<tab>target expression<newline>
    data = [a.split('\t') for a in open(inpfile).readlines()]
    
    X, Y = [], []
    for i, (x, y) in enumerate(data):
        X.append('\t%s\n' % x)
        # get rid of spaces in target sequence
        y = ''.join(y.split())
        Y.append('\t%s\n' % y)
        
    print('Example source:')
    print(X[-1])
    pprint(X[-1])
    print('Example target:')
    print(Y[-1])
    pprint(Y[-1])

    return (X, Y)

def pprint(expr):
    display(eval(expr))

Overwriting seq2sequtil.py


#### Display a few sequence pairs

In [126]:
import seq2sequtil as sq
import importlib
importlib.reload(sq)
inputs, targets = sq.loadData('data/seq2seq_data_10000.txt')

Example source:
	tan(4*x**2-2)/(5*x+1)*sinh(-2*x/7)



    ⎛   2    ⎞     ⎛2⋅x⎞ 
-tan⎝4⋅x  - 2⎠⋅sinh⎜───⎟ 
                   ⎝ 7 ⎠ 
─────────────────────────
         5⋅x + 1         

Example target:
	2*x*tan(2)/7-10*x**2*tan(2)/7+x**3*(7354*tan(2)/1029-8*tan(2)**2/7-8/7)+x**4*(40/7+40*tan(2)**2/7-36770*tan(2)/1029)



   ⎛                         2                  ⎞      ⎛                   2  
 4 ⎜                   40⋅tan (2)   36770⋅tan(2)⎟    3 ⎜7354⋅tan(2)   8⋅tan (2
x ⋅⎜5.71428571428571 + ────────── - ────────────⎟ + x ⋅⎜─────────── - ────────
   ⎝                       7            1029    ⎠      ⎝    1029          7   

                    ⎞       2                    
)                   ⎟   10⋅x ⋅tan(2)   2⋅x⋅tan(2)
─ - 1.14285714285714⎟ - ──────────── + ──────────
                    ⎠        7             7     

### Check data preparer

In [127]:
fractions=[8/10, 9/10]
db = sq.Seq2SeqDataPreparer(inputs, targets, fractions)
print(db)

number of seq-pairs (train):     8000
number of seq-pairs (valid):     1000
number of seq-pairs (test):      1000

number of source tokens:           31
max source sequence length:        81

number of target tokens:           34
max target sequence length:       930


### Check data loader 

In [132]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader = sq.Seq2SeqDataLoader(db.train_data, device)

for i, (X, Y) in enumerate(train_loader):
    print('%5d\t%s\t%s' % (i+1, X.shape, Y.shape))
    if i >= 24: break

    1	torch.Size([25, 5])	torch.Size([85, 5])
    2	torch.Size([46, 2])	torch.Size([106, 2])
    3	torch.Size([62, 1])	torch.Size([171, 1])
    4	torch.Size([62, 1])	torch.Size([42, 1])
    5	torch.Size([42, 1])	torch.Size([329, 1])
    6	torch.Size([47, 1])	torch.Size([265, 1])
    7	torch.Size([34, 1])	torch.Size([33, 1])
    8	torch.Size([55, 2])	torch.Size([300, 2])
    9	torch.Size([57, 1])	torch.Size([69, 1])
   10	torch.Size([24, 3])	torch.Size([48, 3])
   11	torch.Size([13, 2])	torch.Size([18, 2])
   12	torch.Size([38, 1])	torch.Size([137, 1])
   13	torch.Size([35, 2])	torch.Size([93, 2])
   14	torch.Size([68, 1])	torch.Size([281, 1])
   15	torch.Size([64, 3])	torch.Size([187, 3])
   16	torch.Size([51, 1])	torch.Size([102, 1])
   17	torch.Size([24, 1])	torch.Size([54, 1])
   18	torch.Size([22, 1])	torch.Size([45, 1])
   19	torch.Size([59, 1])	torch.Size([170, 1])
   20	torch.Size([44, 1])	torch.Size([286, 1])
   21	torch.Size([67, 1])	torch.Size([269, 1])
   22	torch.Size([53, 